In [1]:
import ee
import wrapper as wp
import png_eds as ed
import geopandas as gpd

ee.Authenticate()

Enter verification code:  4/1AbUR2VO1AqT3yaAlsQVQrtEkJIPIySPu623szRib9Vff1NHBDL6Gm0exKu8



Successfully saved authorization token.


In [2]:
shapefile = gpd.read_file("../../GIS_Data/png_0_2_deg_tiles_Bougainville.shp")


In [3]:
shapefile.tail()

,fid,left,top,right,bottom,id,layer,path,geometry
40,1277.0,155.7,-6.1,155.9,-6.3,3873,png_0_2_deg_tiles,/Users/carole/Documents/Working_Folder/PNGproj...,"POLYGON ((155.70000 -6.10000, 155.90000 -6.100..."
41,1278.0,155.7,-6.3,155.9,-6.5,3874,png_0_2_deg_tiles,/Users/carole/Documents/Working_Folder/PNGproj...,"POLYGON ((155.70000 -6.30000, 155.90000 -6.300..."
42,1279.0,155.7,-6.5,155.9,-6.7,3875,png_0_2_deg_tiles,/Users/carole/Documents/Working_Folder/PNGproj...,"POLYGON ((155.70000 -6.50000, 155.90000 -6.500..."
43,1214.0,154.5,-4.9,154.7,-5.1,3555,png_0_2_deg_tiles,/Users/carole/Documents/Working_Folder/PNGproj...,"POLYGON ((154.50000 -4.90000, 154.70000 -4.900..."
44,1215.0,154.5,-5.1,154.7,-5.3,3556,png_0_2_deg_tiles,/Users/carole/Documents/Working_Folder/PNGproj...,"POLYGON ((154.50000 -5.10000, 154.70000 -5.100..."


In [4]:
def get_tile_coords(id_tile):
    
    loc_id = int(id_tile)
    print("loc_id: ", loc_id)
    geom = shapefile.iloc[loc_id].geometry
    xmin = min(shapefile.iloc[loc_id].geometry.boundary.coords.xy[0])
    xmax = max(shapefile.iloc[loc_id].geometry.boundary.coords.xy[0])
    ymin = min(shapefile.iloc[loc_id].geometry.boundary.coords.xy[1])
    ymax = max(shapefile.iloc[loc_id].geometry.boundary.coords.xy[1])
    
    return xmin, ymin, xmax, ymax


In [5]:
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

NASADEM: NASA NASADEM Digital Elevation 30m


In [6]:
len(shapefile)

45

In [7]:
tile = 0
while tile <= (len(shapefile)-1):
    min_x, min_y, max_x, max_y = get_tile_coords(tile)
    # print(min_x, min_y, max_x, max_y)
    site = ee.Geometry.Rectangle([min_x, min_y, max_x, max_y]);
    # listCoords = ee.Array.cat(site.coordinates(), 1); 
    # xCoords = listCoords.slice(1, 0, 1); 
    # yCoords = listCoords.slice(1, 1, 2); 
    # xMin = xCoords.reduce('min', [0]).get([0,0]);
    # yMin = yCoords.reduce('min', [0]).get([0,0]);
    site_name = str(int(min_x * 10))+str(int(min_y * 10))
    filename = 'Woodyarti_30m_' + site_name
    print(filename)
    #/***************************/ 
    #// MAIN
    #/***************************/ 
    #Parameters
    s1_parameters = {'START_DATE': '2020-01-01',
                'STOP_DATE': '2020-12-31',        
                'POLARIZATION': 'VVVH',
                'PLATFORM_NUMBER': None,
                'ORBIT' : 'ASCENDING',
                'ORBIT_NUM': None,
                'ROI': site,
                'APPLY_BORDER_NOISE_CORRECTION': True,
                'APPLY_SPECKLE_FILTERING': False,
                'SPECKLE_FILTER_FRAMEWORK':None,
                'SPECKLE_FILTER': None,
                'SPECKLE_FILTER_KERNEL_SIZE': None,
                'SPECKLE_FILTER_NR_OF_IMAGES': None,
                'APPLY_TERRAIN_FLATTENING': True,
                'DEM': ee.Image('USGS/SRTMGL1_003'),
                'TERRAIN_FLATTENING_MODEL': 'VOLUME',
                'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
                'FORMAT': 'DB',
                'CLIP_TO_ROI': False,
                'SAVE_ASSET': False,
                'ASSET_ID': None
                }
    #process s1 collection
    s1_processed = wp.s1_preproc(s1_parameters)
    #process woodyarti product
    woody_10m = ed.woodyarti(s1_processed)
    # export
    task_woody = ee.batch.Export.image.toDrive(
      image=woody_10m,
      description=filename,
      folder='GEE_Geotiffs_WoodyArti_Bougainville',
      crs='EPSG:32755',
      scale=30,
      region=site);
    task_woody.start()
    tile = tile+1

loc_id:  0
Woodyarti_30m_1557-69
Selecting ORBIT  ASCENDING
Selecting POLARIZATION  VVVH
Number of images in collection:  29
Additional border noise correction is completed
Radiometric terrain normalization is completed
Convertion to Decibels is completed
loc_id:  1
Woodyarti_30m_1557-71
Selecting ORBIT  ASCENDING
Selecting POLARIZATION  VVVH
Number of images in collection:  29
Additional border noise correction is completed
Radiometric terrain normalization is completed
Convertion to Decibels is completed
loc_id:  2
Woodyarti_30m_1557-73
Selecting ORBIT  ASCENDING
Selecting POLARIZATION  VVVH
Number of images in collection:  29
Additional border noise correction is completed
Radiometric terrain normalization is completed
Convertion to Decibels is completed
loc_id:  3
Woodyarti_30m_1555-75
Selecting ORBIT  ASCENDING
Selecting POLARIZATION  VVVH
Number of images in collection:  29
Additional border noise correction is completed
Radiometric terrain normalization is completed
Convertion t

In [ ]:
# # site = ee.Geometry.Rectangle([144.2, -7.5, 144.4, -7.3]);
# site = ee.Geometry.Rectangle([min_x, min_y, max_x, max_y]);
# listCoords = ee.Array.cat(site.coordinates(), 1); 
# xCoords = listCoords.slice(1, 0, 1); 
# yCoords = listCoords.slice(1, 1, 2); 
# xMin = xCoords.reduce('min', [0]).get([0,0]);
# yMin = yCoords.reduce('min', [0]).get([0,0]);
# site_name = str(int(xMin.getInfo() * 10))+str(int(yMin.getInfo() * 10))
# filename = 'Woodyarti_30m_' + site_name
# print(filename)

In [ ]:
# #/***************************/ 
# #// MAIN
# #/***************************/ 
# #Parameters

# s1_parameters = {'START_DATE': '2020-01-01',
#             'STOP_DATE': '2020-12-31',        
#             'POLARIZATION': 'VVVH',
#             'PLATFORM_NUMBER': None,
#             'ORBIT' : 'DESCENDING',
#             'ORBIT_NUM': None,
#             'ROI': site,
#             'APPLY_BORDER_NOISE_CORRECTION': True,
#             'APPLY_SPECKLE_FILTERING': False,
#             'SPECKLE_FILTER_FRAMEWORK':None,
#             'SPECKLE_FILTER': None,
#             'SPECKLE_FILTER_KERNEL_SIZE': None,
#             'SPECKLE_FILTER_NR_OF_IMAGES': None,
#             'APPLY_TERRAIN_FLATTENING': True,
#             'DEM': ee.Image('USGS/SRTMGL1_003'),
#             'TERRAIN_FLATTENING_MODEL': 'VOLUME',
#             'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
#             'FORMAT': 'DB',
#             'CLIP_TO_ROI': False,
#             'SAVE_ASSET': False,
#             'ASSET_ID': None
#             }
# #processed s1 collection
# s1_processed = wp.s1_preproc(s1_parameters)

In [ ]:
# woody_10m = ed.woodyarti(s1_processed)

In [ ]:
# datasetVH = s1_processed.select('VH');

In [ ]:
# def threshold(image):
#     thresholding = image.gt(BACKSCATTER_LIM)
#     return thresholding

# thresholded_datasetVH = datasetVH.map(threshold)

In [ ]:
# sumVH = thresholded_datasetVH.sum();
# sumVH_months = sumVH.divide(datasetVH.count()).multiply(12);


In [ ]:
# woody = sumVH_months.gt(months_cum);
# woody_10m = woody.setDefaultProjection(woody.projection()).reproject(
#       crs= datasetVH.first().projection(),
#       scale= 10);

In [ ]:
# woody_10m.getInfo()

In [ ]:
# task_woody = ee.batch.Export.image.toDrive(
#   image=woody_10m,
#   description=filename,
#   folder='GEE_Geotiffs_WoodyArti',
#   crs='EPSG:32755',
#   scale=30,
#   region=site);

In [ ]:
# task_woody.start()

In [ ]:
# saltmarsh = ee.Image("JCU/Murray/GIC/global_tidal_wetland_change/2019");
# print(saltmarsh.getInfo())

In [ ]:
# task_saltmarsh = ee.batch.Export.image.toDrive(
#   image=saltmarsh.select('twprobabilityEnd'),
#   description='Test_ee',
#   crs='EPSG:32755',
#   scale=30,
#   region=site)

# task_saltmarsh.start()

In [ ]:
# task_saltmarsh.status()